# Visual Designer (Data Prep)

In this exercise we will be building a pipeline in Azure Machine Learning using the [Visual Designer](https://docs.microsoft.com/azure/machine-learning/concept-designer). Traditionally the Visual Designer is used for training and deploying models. Here we will build a data prep pipeline that get a dataset ready for downstream model scoring. Below you can see a final picture of the data prep pipeline that will be built as part of this exercise.

The pipeline will join two datasets together that consists of the diabetes dataset. We will perform binning on the Age column. After joining the datasets together, we will use the [SQL Transformation](https://docs.microsoft.com/azure/machine-learning/component-reference/apply-sql-transformation) component to demonstrate the flexibility of the Visual Designer by creating an aggregate dataset. The resulting datasets will be landed in the /1-bronze folder of the data lake. Later we will build in a scoring pipeline that will use the result dataset.

![Final data prep pipeline in Visual Designer](./img/vddataprepfinal.png)

## Step 1: Stage data

Let's first upload our source files to the /0-raw layer of the data lake. We will use this as the source for the pipeline.

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

In [ ]:
#TODO: Supply userid value for naming artifacts.
userid = ''

tabular_dataset_name = 'diabetes-data-bronze-' + userid

print(
tabular_dataset_name
)

In [ ]:
from azureml.core import Datastore, Dataset

# Set datastore name where raw diabetes data is stored.
datastore_name = ''

datastore = Datastore.get(ws, datastore_name)
print("Found Datastore with name: %s" % datastore_name)

In [ ]:
from azureml.data.datapath import DataPath

# Upload local csv files to ADLS using AML Datastore.
ds = Dataset.File.upload_directory(src_dir='../data/stage',
           target=DataPath(datastore,  '0-raw/diabetes/' + userid + '/stage/'),
           show_progress=True)

type(ds)

## Step 2: Create target datasets
Register datasets to use as targets for writing data from pipeline.

In [ ]:
diabetes_ds = Dataset.Tabular.from_delimited_files(path=(datastore,'1-bronze/diabetes/' + userid + '/diabetes.csv'),validate=False,infer_column_types=False)
diabetes_ds.register(ws,name=tabular_dataset_name,create_new_version=True)

diabetes_ds = Dataset.Tabular.from_delimited_files(path=(datastore,'1-bronze/diabetes/' + userid + '/diabetes_sql_example.csv'),validate=False,infer_column_types=False)
diabetes_ds.register(ws,name=tabular_dataset_name + '_sql_example',create_new_version=True)

## Step 3: Create new pipeline

In the Azure ML studio, navigate to <b>Designer</b> and press the <b>+</b> button under <b>New pipeline</b>

![Screenshot of AML Studio highlighting the steps described to create a new pipeline](./img/vdnewpipeline.png)

1. In <b>Settings</b> change the compute type to <b>Compute cluster</b> and select the appropriate compute cluster.
1. Name the pipeline in the <b>Draft name</b> field using the convention "pipeline-data-prep-diabetes-<userid>-prod"

![Settings pane with compute settings and draft name fields highlighted](./img/vdsettingpipelinename.png)

1. Open <b>Data Input and Output</b> from the components menu.
2. Drag <b>Import Data</b> onto the canvas.
3. Change the <b>Data source</b> to <b>URL via HTTP</b>
4. Enter the storage url to the <b>patient-age.csv</b> file in the <b>/0-raw</b> folder of the data lake.
5. Validate by pressing <b>Preview schema</b>

![Import data component for patient-age.csv](./img/vdimportpatientage.png)

1. Open <b>Data Input and Output</b> from the components menu.
2. Drag <b>Import Data</b> onto the canvas.
3. Change the <b>Data source</b> to <b>URL via HTTP</b>
4. Enter the storage url to the <b>patient-levels.csv</b> file in the <b>/0-raw</b> folder of the data lake.
5. Validate by pressing <b>Preview schema</b>

![Import data component for patient-levels.csv](./img/vdimportpatientlevels.png)

1. Open <b>Data Transformation</b> from the components menu.
2. Drag <b>Group Data into Bins</b> onto the canvas.
3. Connect <b>Import Data</b> for patient-age.csv.
4. Change <b>Binning mode</b> to <b>Custom Edges</b>.
5. Paste the following value in the <b>Comma-separated list of bin edges</b> field. "1,11,21,31,41,51,61,71,81,91"
6. Select the <b>Age</b> column for <b>Columns to bin</b>.

![Group Data into Bins component settings](./img/vdbindata.png)

1. Open <b>Data Transformation</b> from the components menu.
2. Drag <b>Join Data</b> onto the canvas.
3. Connect <b>Group Data into Bins</b> component using the <b>Quantized dataset: DataFrameDirectory</b> output to <b>Join Data</b> component <b>Left dataset: DataFrameDirectory</b> input.
4. Connect <b>Import Data</b> for patient-levels.csv to <b>Join Data</b> component <b>Right dataset: DataFrameDirectory</b> input.
5. Set the right and left join key columns to <b>Id</b>
6. Leave defaults as shown in screenshot.

![Join data component settings](./img/vdjoindata.png)

1. Open <b>Data Transformation</b> from the components menu.
2. Drag <b>Select Columns in Dataset</b> onto the canvas.
3. Connect <b>Join Data</b> to <b>Select Columns in Dataset</b>.
4. Add the following columns to <b>Select columns</b>. "Id,PatientID,Age,Age_quantized,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree"

![Select Columns in Dataset settings](./img/vdselectcolumns.png)

1. Open <b>Data Input and Output</b> from the components menu.
2. Drag <b>Export Data</b> onto the canvas.
3. Connect <b>Select Columns in Dataset</b> to <b>Export Data</b>.
4. Choose <b>Azure Data Lake Storage Gen2</b> from the <b>Datastore type</b> dropdown.
5. Select the workshop datastore from the <b>Datastore</b> dropdown.
6. Enter the path to the <b>/1-bronze</b> diabetes folder with filename <b>diabetes.csv</b>
7. Choose <b>csv</b> for the <b>File format</b>.

![Export Data component settings for diabetes.csv](./img/vdexportdiabetesdata.png)

1. Open <b>Data Transformation</b> from the components menu.
2. Drag <b>Apply SQL Transformation</b> onto the canvas.
3. Connect <b>Join Data</b> to the <b>Apply SQL Transformation</b> input <b>t1: DataFrameDirectory</b>.
4. Enter the following SQL statement in the <b>SQL query script</b> field.

```sql
SELECT
PatientID
,MAX(BMI) 
FROM t1
GROUP BY PatientID
```
![Apply SQL Transformation component settings](./img/vdsqltransformation.png)


1. Open <b>Data Input and Output</b> from the components menu.
2. Drag <b>Export Data</b> onto the canvas.
3. Connect <b>Apply SQL Transformation</b> to <b>Export Data</b>.
4. Choose <b>Azure Data Lake Storage Gen2</b> from the <b>Datastore type</b> dropdown.
5. Select the workshop datastore from the <b>Datastore</b> dropdown.
6. Enter the path to the <b>/1-bronze</b> diabetes folder with filename <b>diabetes_sql_example.csv</b>
7. Choose <b>csv</b> for the <b>File format</b>.

![Export Data component settings for diabetes.csv](./img/vdexportdiabetessqldata.png)

## Step 4: Submit and Publish pipeline
First submit the pipeline and ensure it runs as expected. Second publish the pipeline endpoint.

1. Press <b>Submit</b>
2. Choose <b>Create New</b> for Experiment.
3. Name the new experiment using this convention. "pipeline-data-prep-diabetes-<'userid'>-prod"
4. Press the <b>Submit</b> button.
5. Monitor the run for completion.

![Set up pipeline run settings](./img/vdsubmitpipeline.png)

1. Verify <b>diabetes.csv</b> and <b>diabetes_sql_example.csv</b> are created after pipeline run in <b>/1-bronze</b> folder.

![screenshot of Storage Explorer showing output files from pipeline run](./img/vdoutputfiles.png)

1. Verify registered datasets recognize the new files

![dataset overview in AML studio with Files in dataset highlighted showing 1](./img/vddatasetoutput.png)

![dataset overview in AML studio explore showing data sample](./img/vdexploredatasetoutput.png)

1. Open the pipeline and press the <b>Publish</b> button.
2. Choose <b>Create new</b> and name the pipeline endpoint the same as the pipeline draft.
3. Press the <b>Publish</b> button.

![The Set up published pipeline menu in the AML Studio Visual Designer](./img/vdpublishpipeline.png)

## The End

This data prep pipeline will be orchestrated using Azure Data Factory with scoring and training pipelines that are published in Module 4. 